In [ ]:
!pip install pandas numpy scikit-learn plotly


In [25]:
%%writefile app.py
import tempfile
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import streamlit as st


# 1. LOAD DATA
# Path to the CSV file
CSV_PATH = "realistic_bioreactor_dataset.csv"


def analyze_batch(csv_path: str):
    df = pd.read_csv(csv_path, parse_dates=["timestamp"]) #Read CSV and parse timestamp as datetime
    df = df.sort_values("timestamp").reset_index(drop=True) # Sort by time and clean up
    df = df.ffill() #Forward-fill missing values

    # Ensure time_hours
    if "time_hours" not in df.columns:
        df["time_hours"] = (
            df["timestamp"] - df["timestamp"].min()
        ).dt.total_seconds() / 3600.0

    # 2. FEATURE ENGINEERING
    df["prod_slope"] = df["product_conc"].diff() / df["time_hours"].diff() # Product slope: d(product_conc)/dt
    df["od_slope"] = df["od600"].diff() / df["time_hours"].diff() # OD slope: d(od600)/dt
    df["prod_slope_smooth"] = df["prod_slope"].rolling(5, min_periods=1).mean() # Smoothed product slope for plateau detection

    # ----- Deviations from setpoints -----
    # Temperature deviation from nominal 30°C
    df["temp_dev"] = df["temp_c"] - 30.0
    df["ph_dev"] = df["pH"] - 6.5 # pH deviation from nominal 6.5
    # Absolute deviations for magnitude-based features
    df["abs_temp_dev"] = df["temp_dev"].abs()
    df["abs_ph_dev"] = df["ph_dev"].abs()

    window = 5
    # Rolling mean and std of OD600
    df["od600_rm5"] = df["od600"].rolling(window, min_periods=1).mean()
    df["od600_rs5"] = df["od600"].rolling(window, min_periods=1).std()
    # Rolling mean and std of temperature
    df["temp_rm5"] = df["temp_c"].rolling(window, min_periods=1).mean()
    df["temp_rs5"] = df["temp_c"].rolling(window, min_periods=1).std()
    # Rolling mean and std of pH
    df["pH_rm5"] = df["pH"].rolling(window, min_periods=1).mean()
    df["pH_rs5"] = df["pH"].rolling(window, min_periods=1).std()

    df["od600_lag1"] = df["od600"].shift(1) # OD600 at previous time step
    df["product_lag1"] = df["product_conc"].shift(1) # Product concentration at previous time step

    #ML : feature matrix, target, train/test split, RandomForest
    FEATURE_COLS = [
        "time_hours",
        "od600",
        "substrate_conc",
        "temp_c",
        "dissolved_o2",
        "pH",
        "prod_slope_smooth",
        "od_slope",
        "temp_dev",
        "ph_dev",
        "abs_temp_dev",
        "abs_ph_dev",
        "od600_rm5",
        "od600_rs5",
        "temp_rm5",
        "temp_rs5",
        "pH_rm5",
        "pH_rs5",
        "od600_lag1",
        "product_lag1",
    ]

    TARGET_COL = "product_conc"

    df_ml = df.dropna(subset=FEATURE_COLS + [TARGET_COL]).reset_index(drop=True) #dropNaNs
    # Build feature matrix X and target vector y
    X = df_ml[FEATURE_COLS].values
    y = df_ml[TARGET_COL].values
    # Train/validation split
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.2, shuffle=True, random_state=42
    )
    # RandomForest regression model
    model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    # Predict on validation set for evaluation
    y_pred = model.predict(X_valid)
    # Compute model error metrics
    mae = mean_absolute_error(y_valid, y_pred)
    r2 = r2_score(y_valid, y_pred)

    df.loc[df_ml.index, "product_pred"] = model.predict(X)

    last_idx = df["product_pred"].last_valid_index()
    predicted_final = float(df.loc[last_idx, "product_pred"])
    actual_final = float(df.loc[last_idx, "product_conc"])

    # 5. RISK SCORE
    # Average deviations from nominal temp and pH
    temp_dev_mean = abs(df["temp_c"].mean() - 30.0)
    ph_dev_mean = abs(df["pH"].mean() - 6.5)
    od_drop_events = (df["od600"].diff() < -0.05).sum()

    total_time = df["time_hours"].iloc[-1] - df["time_hours"].iloc[0]
    if total_time <= 0:
        total_time = 1e-6
    # Approximate oxygen depletion rate across the run
    o2_slope = (df["dissolved_o2"].iloc[0] - df["dissolved_o2"].iloc[-1]) / total_time

    penalty = (
        temp_dev_mean * 10
        + ph_dev_mean * 20
        + od_drop_events * 10
        + max(0, o2_slope - 5) * 5
    )
    # Clamp risk_score into [0, 100], where higher is healthier
    risk_score = float(np.clip(100 - penalty, 0, 100))

    risk_diagnostics = {
        "temp_dev": float(temp_dev_mean),
        "ph_dev": float(ph_dev_mean),
        "od_drop_events": int(od_drop_events),
        "o2_slope": float(o2_slope),
    }

    # 6. HARVEST TIME (generate recommendation)
    plateau = df[(df["time_hours"] > 1) & (df["prod_slope_smooth"] < 0.05)]
    # First time point where slope stays small -> recommended harvest
    if len(plateau) > 0:
        harvest_idx = plateau.index[0]
    # If no plateau is found, default to last time point
    else:
        harvest_idx = df.index[-1]

    harvest_time_hours = float(df.loc[harvest_idx, "time_hours"])
    harvest_timestamp = df.loc[harvest_idx, "timestamp"]

    # ADDITIONAL BIOPROCESS FEATURES
    df["mu"] = df["od600"].pct_change() / df["time_hours"].diff()
    df["mu"] = df["mu"].replace([np.inf, -np.inf], np.nan).fillna(0)

    df["doubling_time"] = np.where(df["mu"] > 0, np.log(2) / df["mu"], np.nan)

    conditions = [
        df["od_slope"] < 0.001,
        df["od_slope"].between(0.001, 0.03),
        df["od_slope"].between(0.03, 0.1),
        df["od_slope"] > 0.1,
    ]
    choices = ["Lag", "Stationary", "Exponential", "Decline"]
    df["growth_phase"] = np.select(conditions, choices, default="Unknown")

    df["oxygen_limitation"] = df["dissolved_o2"] < 10
    df["substrate_empty"] = df["substrate_conc"] < 0.5

    df["productivity"] = df["product_conc"].diff() / df["time_hours"].diff()
    df["productivity"] = df["productivity"].replace(
        [np.inf, -np.inf], np.nan
    ).fillna(0)

    stability_index = 1 / (
        df["temp_c"].std() +
        df["pH"].std() +
        df["od600"].std()
    )

    peak_rate_idx = df["prod_slope"].idxmax()
    time_peak_rate = df.loc[peak_rate_idx, "time_hours"]

    mu_max = float(df["mu"].max())
    dt_min = float(df["doubling_time"].min(skipna=True)) if df["doubling_time"].notna().any() else np.nan
    sub_depletion_time = (
        float(df.loc[df["substrate_empty"], "time_hours"].min())
        if (df["substrate_empty"]).any()
        else None
    )
    average_productivity = float(
        df["product_conc"].iloc[-1] / df["time_hours"].iloc[-1]
    )
    o2_lim_events = int(df["oxygen_limitation"].sum())
    growth_phase_counts = df["growth_phase"].value_counts().to_dict()

    # 9. DASHBOARD – DO NOT fig.show() in Streamlit
    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True,
        subplot_titles=("Product", "OD600", "Temp & pH"),
        vertical_spacing=0.08
    )

    fig.add_trace(go.Scatter(
        x=df["time_hours"], y=df["product_conc"],
        mode="lines+markers", name="Product (actual)"
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df["time_hours"], y=df["product_pred"],
        mode="lines", name="Predicted"
    ), row=1, col=1)

    fig.add_vline(x=harvest_time_hours, line_dash="dash", line_color="red")

    fig.add_trace(go.Scatter(
        x=df["time_hours"], y=df["od600"],
        mode="lines+markers", name="OD600"
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=df["time_hours"], y=df["temp_c"],
        mode="lines", name="Temperature"
    ), row=3, col=1)

    fig.add_trace(go.Scatter(
        x=df["time_hours"], y=df["pH"],
        mode="lines", name="pH"
    ), row=3, col=1)

    fig.update_layout(height=900, width=1000, title_text="Fermentation Dashboard")

    # Instead of printing, RETURN everything to Streamlit
    return {
        "df": df,
        "predicted_final": predicted_final,
        "actual_final": actual_final,
        "mae": mae,
        "r2": r2,
        "risk_score": risk_score,
        "risk_diagnostics": risk_diagnostics,
        "harvest_time_hours": harvest_time_hours,
        "harvest_timestamp": harvest_timestamp,
        "fig": fig,
        "mu_max": mu_max,
        "dt_min": dt_min,
        "sub_depletion_time": sub_depletion_time,
        "average_productivity": average_productivity,
        "o2_lim_events": o2_lim_events,
        "stability_index": float(stability_index),
        "time_peak_rate": float(time_peak_rate),
        "growth_phase_counts": growth_phase_counts,
    }

# ======================
# Simple Domain Chatbot to answer question
# ======================
def fermentation_chatbot(user_msg: str, df):
    """
    A simple Q&A chatbot that answers questions about the fermentation run.
    """

    user_msg = user_msg.lower()

    # Ask about final product / yield
    if "yield" in user_msg or "product" in user_msg:
        final_pred = df["product_pred"].iloc[-1]
        actual = df["product_conc"].iloc[-1]
        return f"The predicted final product concentration is {final_pred:.2f}, and the actual final value is {actual:.2f}."

    # Growth / OD600 question
    if "od600" in user_msg or "growth" in user_msg:
        od_max = df["od600"].max()
        return f"The peak OD600 reached during the run was {od_max:.2f}."

    # Harvest question
    if "harvest" in user_msg:
        return "Harvest time is recommended when the smoothed product slope remains below 0.05."

    # Growth phase
    if "phase" in user_msg:
        dist = df["growth_phase"].value_counts().to_dict()
        return f"Growth phase distribution: {dist}"

    # Oxygen limitation
    if "oxygen" in user_msg or "o2" in user_msg:
        low_o2 = (df["dissolved_o2"] < 10).sum()
        return f"Oxygen limitation occurred at {low_o2} timepoints."

    return "I'm not sure yet — try asking about yield, OD600, harvest time, oxygen limitation, or growth phase."

# ============ STREAMLIT UI ============

st.set_page_config(page_title="Ferma", layout="wide")
st.title("Ferma")

st.sidebar.header("Data source")
uploaded_file = st.sidebar.file_uploader("Upload fermentation CSV", type=["csv"])
use_sample = st.sidebar.checkbox(
    "Use sample dataset (realistic_bioreactor_dataset.csv)",
    value=True,
)

if st.sidebar.button("Run Analysis", type="primary"):
    try:
        if uploaded_file is not None and not use_sample:
            suffix = Path(uploaded_file.name).suffix or ".csv"
            with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
                tmp.write(uploaded_file.getvalue())
                tmp_path = tmp.name
            csv_path = tmp_path
        else:
            csv_path = CSV_PATH

        result = analyze_batch(csv_path)

        st.subheader("Batch Summary")
        c1, c2, c3, c4 = st.columns(4)
        c1.metric("Predicted Final Product", f"{result['predicted_final']:.2f}")
        c2.metric("Actual Final Product", f"{result['actual_final']:.2f}")
        c3.metric("Risk Score (0–100)", f"{result['risk_score']:.1f}")
        c4.metric("Harvest Time (h)", f"{result['harvest_time_hours']:.2f}")
        st.caption(f"Harvest timestamp: {result['harvest_timestamp']}")

        st.subheader("Model Performance")
        m1, m2 = st.columns(2)
        m1.metric("MAE", f"{result['mae']:.4f}")
        m2.metric("R²", f"{result['r2']:.4f}")

         #RAW DATA PREVIEW
        st.subheader("Raw Data Preview")
        st.dataframe(
            result["df"].head(200),   # show up to 200 rows
            use_container_width=True,
            height=400,
        )

        st.subheader("Growth & Process Metrics")
        g1, g2, g3, g4 = st.columns(4)
        g1.metric("Max μ (1/h)", f"{result['mu_max']:.4f}")
        g2.metric("Min Doubling Time (h)", f"{result['dt_min']:.2f}" if not np.isnan(result['dt_min']) else "N/A")
        g3.metric("Stability Index", f"{result['stability_index']:.4f}")
        g4.metric("Avg Productivity (prod/hr)", f"{result['average_productivity']:.3f}")

        g5, g6, g7 = st.columns(3)
        g5.metric("O₂ Limitation Events", f"{result['o2_lim_events']}")
        if result["sub_depletion_time"] is not None:
            g6.metric("Substrate Depletion Time (h)", f"{result['sub_depletion_time']:.2f}")
        else:
            g6.metric("Substrate Depletion Time (h)", "None")
        g7.metric("Time of Peak Production Rate (h)", f"{result['time_peak_rate']:.2f}")

        st.subheader("Growth Phase Distribution (timepoints)")
        st.write(result["growth_phase_counts"])

        st.subheader("Risk Diagnostics")
        st.write(result["risk_diagnostics"])

        st.subheader("Fermentation Dashboard")
        st.plotly_chart(result["fig"], use_container_width=True)

        #  CHATBOT INTERACTION
        st.subheader("Ask Ferma (🤖 Chatbot Assistant)")
        user_q = st.text_input("Ask a question about the fermentation run:")
        if user_q:
          bot_answer = fermentation_chatbot(user_q, result["df"])
          st.markdown(f"**Ferma:** {bot_answer}")


    except Exception as e:
        st.error(f"Error during analysis: {e}")
else:
    st.info("Choose a data source and click **Run Analysis**.")


Overwriting app.py


In [26]:
from pyngrok import ngrok

ngrok.set_auth_token("35WiNn8d7KVxwlFjk3gRoJBBiia_3L4Y2uVVsyT24S9MAdkgw")
ngrok.kill()

!streamlit run app.py --server.port 8501 --server.headless true &>/dev/null &

public_url = ngrok.connect(8501)
public_url



<NgrokTunnel: "https://duckie-romeo-uncommitting.ngrok-free.dev" -> "http://localhost:8501">